In [2]:
import pandas as pd
from data_loading import DataLoader
from preprocess import ConvertToMarkdown, SplitDocuments
from openai_utils import OpenAIEmbeddingClient, GenerateEmbeddings
from pinecone_utils import PineconeIndexClient, IngestEmbeddingsToPinecone
import os
from dotenv import load_dotenv

In [3]:
prova_oab = ConvertToMarkdown(DataLoader('D:\\Users\\Gabriel\\Desktop\\GIT\\pinecone-ingestor\\documentos_ingest\\provas\\40_prova.pdf'))

In [4]:
print(prova_oab)

['“Qualquer semelhança nominal e/ou situacional presente nos enunciados das questões é mera coincidência”\nOrdem dos Advogados do Brasil\n## 40o exame de ordem unificado\n## Tipo 1 – branca\n\uf0b7 Além deste caderno contendo 80 (oitenta) questões\nobjetivas e um questionário de percepção sobre a\nprova com 10 (dez) questões objetivas, você receberá\ndo fiscal de sala:\n\uf09f uma folha destinada às respostas das questões\nobjetivas.\n\uf0b7\n\uf0b7 Verifique se o número deste caderno de provas coincide\ncom o registrado no rodapé de cada página. Caso contrário,\nnotifique imediatamente o fiscal da sala , para que sejam\ntomadas as devidas providências.\n\uf0b7 Confira seus dados pessoais, especialmente nome, número\nde inscrição e documento de identidade e leia atentamente\nas instruções para preencher a folha de respostas.\n\uf0b7 Assinale seu nome, no espaço reservado, com caneta\nesferográfica transparente de cor azul ou preta.\n\uf0b7 Confira se seu tipo/cor de prova corresponde a

In [ ]:
import re

def ProvaOABFormatter(lista_entrada):
    texto = ' '.join(lista_entrada).replace('\n', ' ').strip()

    # Expressão regular para separar questões
    padrao_questoes = re.split(r'(?=\b\d{1,2}\b\s+)', texto)

    questoes_formatadas = []

    for questao in padrao_questoes:
        if not questao.strip():
            continue

        # Separa enunciado das alternativas
        partes = re.split(r'\bA\)\s', questao, maxsplit=1)
        if len(partes) < 2:
            continue

        enunciado = partes[0].strip()
        alternativas_brutas = 'A) ' + partes[1].strip()

        # Captura alternativas A a D (e opcionalmente E se houver)
        alternativas = re.findall(r'([A-D])\)\s(.*?)(?=\s[A-D]\)|$)', alternativas_brutas, re.DOTALL)

        questao_dict = {'pergunta': enunciado}
        for letra, texto_alt in alternativas:
            questao_dict[letra.lower()] = texto_alt.strip()

        questoes_formatadas.append(questao_dict)

    return questoes_formatadas

In [23]:
resultado = separar_questoes(prova_oab)
from pprint import pprint
pprint(resultado[79])

{'a': 'As custas deverão incidir em 2% sobre o valor do acordo e serão '
      'divididas em frações iguais pelas partes, sendo que, no caso de seu '
      'cliente, não haverá o pagamento por força da gratuidade de justiça.',
 'b': 'As custas deverão incidir em 10% sobre o valor do acordo e serão '
      'integralmente recolhidas pela parte ré.',
 'c': 'As custas deverão incidir em 2% sobre o valor do acordo e ficarão '
      'integralmente sob responsabilidade da parte autora que, na hipótese, '
      'está dispensada do recolhimento por força da gratuidade de justiça.',
 'd': 'As custas deverão incidir em 5% sobre o valor da causa, já que não '
      'houve prolação de sentença, e serão rateadas igualmente pelas partes, '
      'dispensado o autor do recolhimento pela gratuidade de justiça. ## '
      'Questionário de percepção sobre a prova Este questionário é de '
      'preenchimento facultativo pelo examinando. Como se trata de mera '
      'pesquisa, não influi no resultado fin

In [97]:
def GabaritoFormatter(gabarito_raw):
    first_page_content = gabarito_raw[0].lower().split("exame de ordem unificado - ")[1]
    first_page_clean = first_page_content.replace('##','').replace('(*) questão anulada', '')
    lines = first_page_clean.split('\n')
    # Removendo lixo no 1o e último itens da lista
    lines = lines[1:-1]
    lines_trimmed = []
    for line in lines:
        line_trimmed = line.strip()
        lines_trimmed.append(line_trimmed)
    
    questions = []
    answers = []
    # Processa a lista em pares: numeração (linha par) e gabarito (linha ímpar)
    for i in range(0, len(lines_trimmed), 2):
        numbers = lines_trimmed[i].split()
        leters = lines_trimmed[i+1].split()
        
        for number, leter in zip(numbers, leters):
            if leter == '*':
                leter = 'ANULADA'
            questions.append(int(number))
            answers.append(leter)

    # Cria o DataFrame
    df = pd.DataFrame({
        'questao': questions,
        'resposta': answers
    })

    # Ordena por número da questão
    df = df.sort_values(by='questao').reset_index(drop=True)
    return df

In [ ]:

gabarito_oab = ConvertToMarkdown(DataLoader('D:\\Users\\Gabriel\\Desktop\\GIT\\pinecone-ingestor\\documentos_ingest\\provas\\40_gabarito.pdf'))

In [96]:
first_page_content = gabarito_oab[0].lower().split("exame de ordem unificado - ")[1]
first_page_clean = first_page_content.replace('##','').replace('(*) questão anulada', '')
lines = first_page_clean.split('\n')
lines = lines[1:-1]
print(lines)

['1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20', ' b c a c * c d a b c b d b b b d a b d a', '21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40', ' d b a c a c d d b c d b d d a c b c c b', '41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60', ' c a d c d c b a a c b d b a d c a b d a', '61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80', ' d a c c a b a b c a b a c d d d c c b a', '']


In [99]:
GabaritoFormatter(gabarito_oab)

IndexError: list index out of range